In [2]:
import pandas as pd
import numpy as np
from functools import reduce
import os
from scipy.interpolate import griddata
from scipy.interpolate.interpolate import interp1d
import plotly.graph_objects as go

In [1]:
def concat_curves():
    merged = pd.DataFrame()
    directory = r'data\diameter'
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            path = os.path.join(directory, file)
            df = pd.read_csv(path)
            df.columns = ['x', 'y']
            df['blades'] = file[:-4].split('_')[1]
            merged = merged.append(df)
    return merged.reset_index(drop=True)

In [3]:
def fill_data(df):
    data = pd.DataFrame()
    x_vals = np.sort(df.x.unique())
    for curve in df.blades.unique():
        trace = df[df.blades == curve]
        y = interp1d(trace.x, trace.y, fill_value='extrapolate')
        data = data.append(pd.DataFrame({'x': x_vals, 'y': y(x_vals), 'blades': curve}), ignore_index=True)
    return data.round(3)

In [9]:
def layout():
    title = {"text": "Maximum Efficiency Curves"}
    xaxis = {'title' : {'text': 'Cn'}}
    yaxis = {'title' : {'text': 'J'}}
    return go.Layout(margin=dict(l=50, r=0, b=0, t=50), 
                     width=1024, height=768, 
                     legend_title_text='Data', title=title, 
                     xaxis=xaxis, yaxis=yaxis)

def draw(df):
    fig = go.Figure(layout = layout())
    for curve in df.blades.unique():
        trace = df[df.blades == curve]
        fig.add_trace(go.Scatter(x=trace.x, y=trace.y, mode='lines', name=curve))
    fig.show()

In [10]:
df = concat_curves()
df = fill_data(df)
draw(df)
df.to_csv('data\\diameter.csv', index_label='id')


In [16]:
df_pivot = df.pivot(index='blades', columns='x', values='y')
df_pivot.to_csv('data\\diameter_pivot.csv', index=False)